# Dữ liệu dạng bảng

In [1]:
import numpy as np
import pandas as pd

In [3]:
my_data = pd.read_csv("data/drug200.csv", delimiter=",")
my_data.head()

,Age,Sex,BP,Cholesterol,Na_to_K,Drug
0,23,F,HIGH,HIGH,25.355,drugY
1,47,M,LOW,HIGH,13.093,drugC
2,47,M,LOW,HIGH,10.114,drugC
3,28,F,NORMAL,HIGH,7.798,drugX
4,61,F,LOW,HIGH,18.043,drugY


In [5]:
my_data.shape

(200, 6)

In [8]:
# values để chuyển data frame sang numpy dùng cho thư viện sklearn
X = my_data[['Age', 'Sex', 'BP', 'Cholesterol', 'Na_to_K']].values
X[0:5]

array([[23, 'F', 'HIGH', 'HIGH', 25.355],
       [47, 'M', 'LOW', 'HIGH', 13.093],
       [47, 'M', 'LOW', 'HIGH', 10.114],
       [28, 'F', 'NORMAL', 'HIGH', 7.798],
       [61, 'F', 'LOW', 'HIGH', 18.043]], dtype=object)

Chuyển từ categorical sang số thực

In [13]:
from sklearn import preprocessing
le_sex = preprocessing.LabelEncoder()
X[:,1] = le_sex.fit_transform(X[:,1])

le_BP = preprocessing.LabelEncoder()
X[:,2] = le_BP.fit_transform(X[:,2])

le_Chol = preprocessing.LabelEncoder()
X[:,3] = le_Chol.fit_transform(X[:,3])

X[0:5]

array([[23, 0, 0, 0, 25.355],
       [47, 1, 1, 0, 13.093],
       [47, 1, 1, 0, 10.114],
       [28, 0, 2, 0, 7.798],
       [61, 0, 1, 0, 18.043]], dtype=object)

In [16]:
mean = X[:, -1].mean()
std = X[:, -1].std()
X[:, -1] = (X[:, -1] - mean) / std

X[0:5]

array([[23, 0, 0, 0, 1.2865221173753494],
       [47, 1, 1, 0, -0.4151453955143373],
       [47, 1, 1, 0, -0.8285581765368765],
       [28, 0, 2, 0, -1.1499626749753462],
       [61, 0, 1, 0, 0.27179427083731394]], dtype=object)

In [18]:
y = my_data["Drug"]
y[0:5]

0    drugY
1    drugC
2    drugC
3    drugX
4    drugY
Name: Drug, dtype: object

In [20]:
y = preprocessing.LabelEncoder().fit_transform(y)
y[0:5]

array([4, 2, 2, 3, 4], dtype=int64)

# TI_DF

## Phương pháp mã hóa: TF-IDF
Cho tập gồm $n$ văn bản: $D = \{d_1, d_2, ... d_n\}$. Tập từ điển tương ứng được xây dựng từ $n$ văn bản này có độ dài là $m$
- Xét văn bản $d$ có $|d|$ từ và $t$ là một từ trong $d$. Mã hóa tf-idf của $t$ trong văn bản $d$ được biểu diễn:
\begin{equation}
    \begin{split}
        \text{tf}_{t, d} &= \frac{f_t}{|d|} \\
        \text{idf}_{t, d} &= \log\frac{n}{n_t}, \ \ \ \ n_t = |\{d\in D: t\in d\}| \\
        \text{tf-idf}_{t d} &= \text{tf}_{t, d} \times \text{idf}_{t, d}
    \end{split}
\end{equation}

- Khi đó văn bản $d$ được mã hóa là một vector $m$ chiều. Các từ xuất hiện trong d sẽ được thay bằng giá trị tf-idf tương ứng. Các từ không xuất hiện trong $d$ thì thay là 0

In [26]:
import numpy as np
print(np.testing)

<module 'numpy.testing' from 'C:\\Users\\nguye\\miniconda3\\envs\\course\\lib\\site-packages\\numpy\\testing\\__init__.py'>


In [2]:
import os
import matplotlib.pyplot as plt
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.datasets import load_files

%matplotlib inline

In [3]:
INPUT = 'data/news_vnexpress'
os.makedirs("images",exist_ok=True)  # thư mục lưu các các hình ảnh kết quả trong quá trình huấn luyện và đánh giá

In [4]:
print('Các nhãn và số văn bản tương ứng trong dữ liệu')
print('----------------------------------------------')
n = 0
for label in os.listdir(INPUT):
    print(f'{label}: {len(os.listdir(os.path.join(INPUT, label)))}')
    n += len(os.listdir(os.path.join(INPUT, label)))

print('-------------------------')
print(f"Tổng số văn bản: {n}")

Các nhãn và số văn bản tương ứng trong dữ liệu
----------------------------------------------
doi-song: 120
du-lich: 54
giai-tri: 201
giao-duc: 105
khoa-hoc: 144
kinh-doanh: 262
phap-luat: 59
suc-khoe: 162
the-thao: 173
thoi-su: 59
-------------------------
Tổng số văn bản: 1339


In [6]:
data_train = load_files(container_path=INPUT, encoding='utf-8')
for i in range(len(data_train.target_names)):
    print(f'{data_train.target_names[i]} - {i}')

print('--------------------------')
print(data_train.filenames[0:1])
# print(data_train.data[0:1])
print(data_train.target[0:1])
print(data_train.data[0:1])

print("\nTổng số  văn bản: {}" .format( len(data_train.filenames)))

doi-song - 0
du-lich - 1
giai-tri - 2
giao-duc - 3
khoa-hoc - 4
kinh-doanh - 5
phap-luat - 6
suc-khoe - 7
the-thao - 8
thoi-su - 9
--------------------------
['data/news_vnexpress\\khoa-hoc\\00133.txt']
[4]
['Mời độc giả đặt câu hỏi tại đây\n']

Tổng số  văn bản: 1339


In [10]:
with open("data/vietnamese-stopwords.txt", encoding="utf8") as f:
    stopwords = f.readlines()
stopwords = [x.strip().replace(" ", "_") for x in stopwords]
print(f"Số lượng stopwords: {len(stopwords)}")
print(stopwords[:10])

module_count_vector = CountVectorizer(stop_words=stopwords)
model_rf_prepreocess = Pipeline([
    ('vect', module_count_vector),
    ('tfidf', TfidfTransformer()),
])

data_preprocessed = model_rf_prepreocess.fit_transform(data_train.data, data_train.target)
print(f"\nSố lượng từ trong từ điển: {len(module_count_vector.vocabulary_)}")
print(f"Kích thước dữ liệu sau khi xử lý: {data_preprocessed.shape}")
print(f"Kích thước nhãn tương ứng: {data_train.target.shape}")


Số lượng stopwords: 2063
['a_lô', 'a_ha', 'ai', 'ai_ai', 'ai_nấy', 'ai_đó', 'alô', 'amen', 'anh', 'anh_ấy']

Số lượng từ trong từ điển: 12796
Kích thước dữ liệu sau khi xử lý: (1339, 12796)
Kích thước nhãn tương ứng: (1339,)


In [11]:
X = data_preprocessed
Y = data_train.target

In [12]:
X.shape, Y.shape

((1339, 12796), (1339,))

In [13]:
print(X[100].toarray())
print(Y[100])

[[0.         0.         0.         ... 0.         0.14048828 0.        ]]
5


In [17]:
sum(sum(X[100].toarray() != 0))

289

In [18]:
print(X[100])

  (0, 12794)	0.14048828324700804
  (0, 12724)	0.051226678060487627
  (0, 12714)	0.034379239518190156
  (0, 12705)	0.024927343279465615
  (0, 12697)	0.03935911209707954
  (0, 12692)	0.013885134230282647
  (0, 12691)	0.02076954755505395
  (0, 12672)	0.03173992101554847
  (0, 12646)	0.04268947993761032
  (0, 12643)	0.030193779677554416
  (0, 12629)	0.024173036345759045
  (0, 12626)	0.01928809379275951
  (0, 12624)	0.3318224864003995
  (0, 12617)	0.08000423234784886
  (0, 12591)	0.07519534686809994
  (0, 12584)	0.03876774373554222
  (0, 12566)	0.033240367004725005
  (0, 12558)	0.03206234356763185
  (0, 12547)	0.04575286598942787
  (0, 12535)	0.05488370325838488
  (0, 12521)	0.09355442947181113
  (0, 12517)	0.03883219864696093
  (0, 12509)	0.017786174579851665
  (0, 12454)	0.07589970050190288
  (0, 12272)	0.02125953768208212
  :	:
  (0, 2170)	0.029508397725910254
  (0, 2159)	0.016084504788746505
  (0, 2140)	0.015661963686282587
  (0, 2135)	0.04322051581452054
  (0, 2111)	0.02577563465433511